In [11]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/SPIRS")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#USER HISTORY

In [12]:
OUTPUT_FILE_HISTORY = 'user_embeddings_non_sarcastic_history_50.csv'   #file in which we write user history
INPUT_FILE = 'spirs_history/SPIRS-non-sarcastic-history.txt'
TOKENS_NO = 50

In [19]:
import random
import nltk
import zipfile
import csv
from csv import DictWriter
from utils.preprocessing import process_tweet

os.chdir("/content/drive/My Drive/SP")
csv_file = open(OUTPUT_FILE_HISTORY, 'a')
dictwriter_object = DictWriter(csv_file, fieldnames=['user_id', 'history'])

os.chdir("/content/drive/My Drive/SPIRS")
with zipfile.ZipFile('spirs_history.zip') as zip:
    with zip.open(INPUT_FILE, mode='r') as file:
      line = file.readline().split(b'\t')
      old_user_id = int(line[0])

with zipfile.ZipFile('spirs_history.zip') as zip:
    with zip.open(INPUT_FILE, mode='r') as file:
      sar_text = []
      for line in file:
        line = line.decode('UTF-8')
        line = line.split('\t')
        user_id = int(line[0])
        
        tweet_text = process_tweet(line[2]) 
        
        if (len(tweet_text)  == 0):
          continue
        
        if user_id == old_user_id: 
          sar_text.append(tweet_text)
        else:
          sar_text = random.sample(sar_text, len(sar_text))
          sentences = ''
          count = 0
          #count tokens, add tweets to embedding 
          for tweet in sar_text:
            tokens = nltk.word_tokenize(tweet)
            count += len(tokens) + 1 #+1 because of [SEP]
            if count <= TOKENS_NO:
              sentences += tweet + ' [SEP] '
            else:
              count -= len(tokens)
              sar_text = []
              break

          #if num of tokens is smaller than expected add padding
          while count <= TOKENS_NO:
            sentences += ' [PAD]'
            count += 1

          embedding = {'user_id': old_user_id, 'history': sentences}
          #dictwriter_object.writerow(embedding)

          old_user_id = user_id
          

Streaming output truncated to the last 5000 lines.
oil mixed rally falters supply concerns, vaccine doubts
australia says 13 soldiers told face dismissal afghan report
clashes erupted argentine police fans lined casket soccer icon diego maradona
huawei founder urges honor subbrand competitor split
china's industrial profits rise 28.2% year-on-year october
u.s. president-elect joe biden wife released video acknowledging scaled-back thanksgiving celebrations transformed coronavirus pandemic
breakingviews - indonesia builds sovereign fund daring design
gaining access vault frank zappa's private videos, alex winter spent years turning material documentary 'zappa,' presents unique portrait influential late musician
china impose temporary anti-dumping measures australian wine imports
dyson invest 2.7 billion double product range 2025
rapid, lab-free testing process given members london symphony orchestra confidence return rehearsals
south korea braces bed shortages coronavirus cases near nin

KeyboardInterrupt: ignored

#CONCAT USER HISTORY WITH TWEETS

In [ ]:
!pip install emoji

In [ ]:
from csv import reader
import csv
import pandas as pd
import numpy as np
from utils.preprocessing import remove_na_from_column, preprocess_tweets

os.chdir("/content/drive/My Drive/SPIRS")

INPUT_FILE = 'SPIRS-non-sarcastic.csv'

df = pd.read_csv(INPUT_FILE)
df = pd.DataFrame({'sar_user': np.array(df['sar_user']), 'sar_id': np.array(df['sar_id']), 'sar_text': np.array(df['sar_text'])})
df = remove_na_from_column(df, 'sar_user')
df = remove_na_from_column(df, 'sar_id')
df = remove_na_from_column(df, 'sar_text')
df = preprocess_tweets(df, 'sar_text')

for i in range(len(df['sar_user'])):
  user = df['sar_user'][i].split('|')
  df['sar_user'][i] = int(user[-1])

In [ ]:
os.chdir("/content/drive/My Drive/SP")

INPUT_FILE_HISTORY = 'user_embeddings_non_sarcastic_history_50.csv'
OUTPUT_FILE_EMBEDDING = 'user_embeddings_non_sarcastic_end_50.csv'

file_history = open(INPUT_FILE_HISTORY, 'r')
csv_reader = reader(file_history)

file_write = open(OUTPUT_FILE_EMBEDDING, 'a')
writer = csv.writer(file_write)
writer.writerow(['user_id', 'sar_id', 'sar_text'])

end = True   #if true add tweet to the end

for row in csv_reader:
  users_tweets = df[df['sar_user'] == int(row[0])]   #find tweets of user
  sar_id = users_tweets['sar_id'].values
  tweets = users_tweets['sar_text'].values

  for i in range(len(tweets)):
    if end:
      extended_tweet = '[CLS] ' + row[1] + ' ' + tweets[i]
    else:
      extended_tweet = '[CLS] ' + tweets[i] + ' [SEP] ' + row[1] 
    
    writer.writerow([str(row[0]), str(sar_id[i]), extended_tweet])   #user_id, sar_id, tweet + history